In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import plotly.express as px

In [2]:
df=pd.read_csv('Stock.csv')

In [3]:
df=df.drop(df[df['Closing Price']==0].index)
df=df[[ 'Date', 'Traded Companies',
       'No. Of Transaction', 'Max Price', 'Min Price', 'Closing Price',
       'Traded Shares', 'Amount', 'Previous Closing']]
print(df.shape)
df=df.drop_duplicates()
df.shape
df['Date']=pd.to_datetime(df['Date'])
df[df['Date']=='2022-04-28']

(10647, 9)


,Date,Traded Companies,No. Of Transaction,Max Price,Min Price,Closing Price,Traded Shares,Amount,Previous Closing
10182,2022-04-28,ADBL,217,378.0,367.1,369.0,26491.0,9776276.7,371.0
10183,2022-04-28,NBL,359,322.0,315.0,315.5,53053.0,16805321.5,318.0
10184,2022-04-28,NMB,225,295.0,292.0,293.0,25873.0,7578511.7,294.2
10185,2022-04-28,NLIC,370,950.0,933.8,934.0,22681.0,21266520.9,938.1
10186,2022-04-28,AHPC,577,431.9,416.9,417.2,111687.0,47334634.2,422.0


In [36]:
fig=px.line(df[df['Traded Companies']=='NBL'],x='Date',y='Closing Price',title="Line Plot of NBL's Close Price ")
fig.show()

In [5]:
x=df[[ 'Max Price', 'Min Price',
       'Traded Shares', 'Amount']]
y=df[['Closing Price']]


In [6]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,shuffle=False)

In [7]:
scaler=StandardScaler()
scaler.fit(x)
x_train=scaler.transform(x_train)
x_test=scaler.transform(x_test)

In [8]:
y_scaler=StandardScaler()
y_scaler.fit(y)
y_train=y_scaler.transform(y_train)

In [9]:
lr=LinearRegression()

In [10]:
lr.fit(x_train,y_train.flatten())

LinearRegression()

In [11]:
predict=lr.predict(x_test)
predictlr=y_scaler.inverse_transform(predict)

In [12]:
final_data=df[8510:]
final_data['Predict']=predictlr

/var/folders/dd/21_xynmd0yb0fq9p3j2y1_wm0000gn/T/ipykernel_8279/1530910766.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [13]:
final_data=final_data[['Date', 'Traded Companies', 'Closing Price','Predict']]

In [31]:
fig=px.line(final_data[final_data['Traded Companies']=='NBL'],x='Date',y=['Closing Price','Predict'],title="Predicted vs Acutal Closed Price of NBL by using Linear Regression")
fig.show()

In [16]:
print("Test RMSE: ",round(np.sqrt(metrics.mean_squared_error(y_test,predictlr)),2))
print("Test MAE: ",round(metrics.mean_absolute_error(y_test,predictlr),2))
print("Test R-squared: ",round(metrics.r2_score(y_test,predictlr),2))

Test RMSE:  11.63
Test MAE:  6.63
Test R-squared:  1.0


In [17]:
svr=SVR()

In [18]:
svr.fit(x_train,y_train)

/Users/sudhan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



SVR()

In [19]:
predict_svr=svr.predict(x_test)

In [20]:
predict_svr=y_scaler.inverse_transform(predict_svr)

In [21]:
final_data_svr=df[8510:]
final_data_svr['Predicted']=predict_svr


/var/folders/dd/21_xynmd0yb0fq9p3j2y1_wm0000gn/T/ipykernel_8279/2023381745.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [33]:
fig=px.line(final_data_svr[final_data_svr['Traded Companies']=='NBL'],x='Date',y=['Closing Price','Predicted'],title="Predicted vs Acutal Closed Price of NBL by SVR")
fig.show()

In [28]:
print("RMSE: ",round(np.sqrt(metrics.mean_squared_error(y_test,predict_svr)),2))
print("MAE: ",round(metrics.mean_absolute_error(y_test,predict_svr),2))
print("R-squared: ",round(metrics.r2_score(y_test,predict_svr),2))

RMSE:  173.78
MAE:  75.35
R-squared:  0.9
